In [2]:
import yfinance as yf

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='100d', interval='1d')
    data.reset_index(inplace=True, drop=True)
    return data
# Get the data
data = get_data('BTC-USD')

[*********************100%***********************]  1 of 1 completed


In [3]:
import pandas_ta as ta

def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

## 1 - Slope of MAs

In [4]:
import numpy as np

def calculate_slope(series, period: int = 5): #average slop on a period of 5 candles
    slopes = [0 for _ in range(period-1)] #starts from zero, 
    for i in range(period-1, len(series)):
        x = np.arange(period) # 0 to 4, this means for each 5 candles before the last candle
        y = series[i-period+1:i+1].values #slice moving average 
        slope = np.polyfit(x, y, 1)[0]  # Calculate the slope using linear regression, The slope is a measure of how steep the line is. If the slope is positive, it means that the line is going up, and if it is negative, it means that the line is going down. The [0] at the end of the line extracts the slope value from the output of the function.
        percent_slope = (slope / y[0]) * 100  # Convert the slope to a percentage
        slopes.append(percent_slope)
    return slopes

In [5]:
# Calculate the slope
data['Slope'] = calculate_slope(data['SMA'])

In [6]:
data[40:55]

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope
59,37721.414062,37892.429688,37617.417969,37796.792969,37796.792969,9099571165,36725.498047,0.360036
60,37796.828125,37820.300781,37162.750000,37479.121094,37479.121094,13744796068,36847.585547,0.359976
61,37454.191406,37559.355469,36750.128906,37254.167969,37254.167969,19002925720,36938.115820,0.335539
62,37247.992188,38368.480469,36891.089844,37831.085938,37831.085938,21696137014,37046.906250,0.308892
63,37826.105469,38366.113281,37612.632812,37858.492188,37858.492188,20728546658,37105.174609,0.261038
64,37861.117188,38141.753906,37531.140625,37712.746094,37712.746094,18115982627,37125.113477,0.195973
65,37718.007812,38954.109375,37629.359375,38688.750000,38688.750000,23512784002,37202.648438,0.164403
66,38689.277344,39678.937500,38652.593750,39476.332031,39476.332031,15534035612,37323.739062,0.175761
67,39472.207031,40135.605469,39298.164062,39978.390625,39978.390625,15769696322,37497.540820,0.265019
68,39978.628906,42371.750000,39978.628906,41980.097656,41980.097656,39856129827,37819.663672,0.453599


In [7]:
import plotly.graph_objects as go

dfpl = data[:] # all the data frame
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['SMA'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [8]:
data

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope
19,27162.628906,29448.138672,27130.472656,28519.466797,28519.466797,27833876539,27363.613281,0.000000
20,28522.097656,28618.751953,28110.185547,28415.748047,28415.748047,14872527508,27466.764844,0.000000
21,28413.531250,28889.009766,28174.251953,28328.341797,28328.341797,12724128586,27532.104590,0.000000
22,28332.416016,28892.474609,28177.988281,28719.806641,28719.806641,14448058195,27622.508887,0.000000
23,28732.812500,30104.085938,28601.669922,29682.949219,29682.949219,21536125230,27758.260547,0.345363
...,...,...,...,...,...,...,...,...
95,42152.097656,42860.937500,41998.253906,42265.187500,42265.187500,16397498810,42739.712695,-0.125593
96,42280.234375,44175.437500,42214.976562,44167.332031,44167.332031,18426978443,42875.568164,0.004396
97,44187.140625,45899.707031,44176.949219,44957.968750,44957.968750,39335274536,42978.929492,0.141505
98,44961.601562,45503.242188,40813.535156,42848.175781,42848.175781,46342323118,42970.139648,0.187920


## 2 - 3 MAs alignment

In [9]:
# Calculate the moving averages
data['SMA_10'] = calculate_sma(data, 10)
data['SMA_20'] = calculate_sma(data, 20)
data['SMA_30'] = calculate_sma(data, 30)

In [10]:
data

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_30
19,27162.628906,29448.138672,27130.472656,28519.466797,28519.466797,27833876539,27363.613281,0.000000,NaN,NaN,NaN
20,28522.097656,28618.751953,28110.185547,28415.748047,28415.748047,14872527508,27466.764844,0.000000,NaN,NaN,NaN
21,28413.531250,28889.009766,28174.251953,28328.341797,28328.341797,12724128586,27532.104590,0.000000,NaN,NaN,NaN
22,28332.416016,28892.474609,28177.988281,28719.806641,28719.806641,14448058195,27622.508887,0.000000,NaN,NaN,NaN
23,28732.812500,30104.085938,28601.669922,29682.949219,29682.949219,21536125230,27758.260547,0.345363,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,42152.097656,42860.937500,41998.253906,42265.187500,42265.187500,16397498810,42739.712695,-0.125593,42947.930859,42739.712695,42675.498307
96,42280.234375,44175.437500,42214.976562,44167.332031,44167.332031,18426978443,42875.568164,0.004396,42964.873828,42875.568164,42831.864974
97,44187.140625,45899.707031,44176.949219,44957.968750,44957.968750,39335274536,42978.929492,0.141505,43086.716406,42978.929492,42997.850911
98,44961.601562,45503.242188,40813.535156,42848.175781,42848.175781,46342323118,42970.139648,0.187920,43069.922266,42970.139648,43026.786849


In [11]:
def determine_trend(data):
    if data['SMA_10'] > data['SMA_20'] > data['SMA_30']:
        return 2  # Uptrend
    elif data['SMA_10'] < data['SMA_20'] < data['SMA_30']:
        return 1  # Downtrend
    else:
        return 0  # No trend

# Determine the trend and add it as a new column to the DataFrame
data['Trend'] = data.apply(determine_trend, axis=1)

In [30]:
data[10:40]

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category,ADX,DMP,DMN,Trend Signal,Confirmed Signal
Datetime,,,,,,,,,,,,,
2023-12-29 02:30:00,42343.449219,42473.566406,42343.449219,42460.937500,42460.937500,66545664,42389.535113,1,NaN,NaN,NaN,0,0.0
2023-12-29 02:45:00,42469.199219,42614.644531,42469.199219,42591.691406,42591.691406,200912896,42418.417069,0,NaN,NaN,NaN,0,0.0
2023-12-29 03:00:00,42592.023438,42608.601562,42553.460938,42553.460938,42553.460938,0,42418.417069,0,NaN,NaN,NaN,0,0.0
2023-12-29 03:15:00,42547.933594,42601.562500,42492.343750,42601.562500,42601.562500,0,42418.417069,0,NaN,NaN,NaN,0,0.0
2023-12-29 03:30:00,42602.320312,42777.187500,42602.320312,42755.773438,42755.773438,198645760,42460.823862,0,NaN,31.667138,23.886220,0,0.0
2023-12-29 03:45:00,42747.230469,42784.234375,42729.582031,42729.582031,42729.582031,169211904,42492.288347,2,NaN,30.814350,22.801165,0,0.0
2023-12-29 04:00:00,42717.867188,42780.472656,42694.058594,42701.226562,42701.226562,28139520,42496.459955,2,NaN,28.601966,24.115573,0,0.0
2023-12-29 04:15:00,42696.738281,42699.984375,42632.546875,42636.371094,42636.371094,0,42496.459955,2,NaN,26.946076,27.904616,0,0.0
2023-12-29 04:30:00,42638.328125,42665.402344,42633.027344,42657.652344,42657.652344,0,42496.459955,2,NaN,26.176738,27.107910,0,0.0


In [13]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_10'], mode='lines', name='SMA 10', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='lines', name='SMA 20', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_30'], mode='lines', name='SMA 30', line=dict(color='green')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 3 - Candles above or below the MA curve

In [14]:
#another way to evaluate the trend, for instance 5 candle are up the moving average and will be  an uptrend 
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['Close'][i-backcandles:i] > data[ma_column][i-backcandles:i]): #if closing candles are above moving average then uptrend
            categories.append(2)  # Uptrend
        elif all(data['Close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Category'] = check_candles(data, 5, 'SMA_20')


In [15]:
data[25:55]

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_30,Trend,Category
44,36702.250000,37493.800781,36362.753906,37313.968750,37313.968750,22711265155,34636.590137,1.066704,35538.267578,34636.590137,NaN,0,2
45,37310.070312,37407.093750,36773.667969,37138.050781,37138.050781,13924272142,34993.797852,1.063114,35708.347266,34993.797852,NaN,0,2
46,37133.992188,37227.691406,36779.117188,37054.519531,37054.519531,11545715999,35192.212109,0.966734,35919.975000,35192.212109,NaN,0,2
47,37070.304688,37405.117188,36399.605469,36502.355469,36502.355469,19057712790,35322.253516,0.778159,36096.978125,35322.253516,NaN,0,2
48,36491.789062,36753.351562,34948.500000,35537.640625,35537.640625,23857403554,35373.994531,0.520624,36142.522656,35373.994531,33751.636458,2,2
49,35548.113281,37964.894531,35383.781250,37880.582031,37880.582031,27365821679,35560.191211,0.375658,36425.645312,35560.191211,34063.673633,2,2
50,37879.980469,37934.625000,35545.472656,36154.769531,36154.769531,26007385366,35672.439648,0.340528,36537.385156,35672.439648,34321.641016,2,2
51,36164.824219,36704.484375,35901.234375,36596.683594,36596.683594,22445028430,35797.795117,0.353751,36652.697266,35797.795117,34597.252409,2,2
52,36625.371094,36839.281250,36233.312500,36585.703125,36585.703125,11886022717,35900.156250,0.364654,36745.739844,35900.156250,34859.448958,2,2
53,36585.765625,37509.355469,36414.597656,37386.546875,37386.546875,12915986553,36044.365430,0.336349,36815.082031,36044.365430,35116.235547,2,2


In [16]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='lines', name='SMA 20', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 4 - Apply trend detection using the VWAP curve

In [17]:
# Download the BTC-USD 15 min data for the last 7 days
data = yf.download('BTC-USD', period='7d', interval='15m')
# Compute the VWAP
data.ta.vwap(append=True)

[*********************100%***********************]  1 of 1 completed


Datetime
2023-12-29 00:00:00             NaN
2023-12-29 00:15:00    42671.570312
2023-12-29 00:30:00    42671.570312
2023-12-29 00:45:00    42538.234522
2023-12-29 01:00:00    42453.073347
                           ...     
2024-01-04 17:00:00    43393.163065
2024-01-04 17:15:00    43393.163065
2024-01-04 17:30:00    43403.506682
2024-01-04 17:45:00    43412.968206
2024-01-04 18:00:00    43412.968206
Name: VWAP_D, Length: 649, dtype: float64

In [18]:
# Apply the check_candles function (5 candles above) 
data['Category'] = check_candles(data, 5, 'VWAP_D') 

In [19]:
data[data["Category"]!=0] #find the ones that are a trend

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category
Datetime,,,,,,,,
2023-12-29 02:30:00,42343.449219,42473.566406,42343.449219,42460.937500,42460.937500,66545664,42389.535113,1
2023-12-29 03:45:00,42747.230469,42784.234375,42729.582031,42729.582031,42729.582031,169211904,42492.288347,2
2023-12-29 04:00:00,42717.867188,42780.472656,42694.058594,42701.226562,42701.226562,28139520,42496.459955,2
2023-12-29 04:15:00,42696.738281,42699.984375,42632.546875,42636.371094,42636.371094,0,42496.459955,2
2023-12-29 04:30:00,42638.328125,42665.402344,42633.027344,42657.652344,42657.652344,0,42496.459955,2
...,...,...,...,...,...,...,...,...
2024-01-04 17:00:00,43962.742188,44159.847656,43914.175781,44159.847656,44159.847656,3039232,43393.163065,2
2024-01-04 17:15:00,44196.511719,44245.570312,44128.683594,44128.683594,44128.683594,0,43393.163065,2
2024-01-04 17:30:00,44144.972656,44172.367188,43931.804688,43931.804688,43931.804688,63057920,43403.506682,2


In [20]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['VWAP_D'], mode='lines', name='VWAP', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [21]:
data[:50]

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category
Datetime,,,,,,,,
2023-12-29 00:00:00,42614.644531,42701.996094,42614.644531,42701.996094,42701.996094,0,NaN,0
2023-12-29 00:15:00,42698.023438,42711.757812,42629.710938,42673.242188,42673.242188,9170944,42671.570312,0
2023-12-29 00:30:00,42670.347656,42670.757812,42573.441406,42617.433594,42617.433594,0,42671.570312,0
2023-12-29 00:45:00,42633.335938,42647.617188,42443.046875,42443.046875,42443.046875,45293568,42538.234522,0
2023-12-29 01:00:00,42442.003906,42529.808594,42273.753906,42517.230469,42517.230469,362108928,42453.073347,0
2023-12-29 01:15:00,42502.250000,42562.734375,42356.277344,42423.562500,42423.562500,65460224,42452.319845,0
2023-12-29 01:30:00,42426.253906,42426.253906,42216.687500,42278.179688,42278.179688,178071552,42413.128907,0
2023-12-29 01:45:00,42325.976562,42357.656250,42219.582031,42357.656250,42357.656250,219957248,42387.761290,0
2023-12-29 02:00:00,42375.054688,42381.722656,42309.988281,42330.921875,42330.921875,0,42387.761290,0


## 5 - Trend confirmation using the ADX

In [22]:
# Calculate the ADX
data.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
Datetime,,,
2023-12-29 00:00:00,NaN,NaN,NaN
2023-12-29 00:15:00,NaN,NaN,NaN
2023-12-29 00:30:00,NaN,NaN,NaN
2023-12-29 00:45:00,NaN,NaN,NaN
2023-12-29 01:00:00,NaN,NaN,NaN
...,...,...,...
2024-01-04 17:00:00,38.682119,41.384798,13.199419
2024-01-04 17:15:00,39.844647,43.047066,12.512765
2024-01-04 17:30:00,39.317199,38.596834,19.679967


In [23]:
data

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category,ADX_14,DMP_14,DMN_14
Datetime,,,,,,,,,,,
2023-12-29 00:00:00,42614.644531,42701.996094,42614.644531,42701.996094,42701.996094,0,NaN,0,NaN,NaN,NaN
2023-12-29 00:15:00,42698.023438,42711.757812,42629.710938,42673.242188,42673.242188,9170944,42671.570312,0,NaN,NaN,NaN
2023-12-29 00:30:00,42670.347656,42670.757812,42573.441406,42617.433594,42617.433594,0,42671.570312,0,NaN,NaN,NaN
2023-12-29 00:45:00,42633.335938,42647.617188,42443.046875,42443.046875,42443.046875,45293568,42538.234522,0,NaN,NaN,NaN
2023-12-29 01:00:00,42442.003906,42529.808594,42273.753906,42517.230469,42517.230469,362108928,42453.073347,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-04 17:00:00,43962.742188,44159.847656,43914.175781,44159.847656,44159.847656,3039232,43393.163065,2,38.682119,41.384798,13.199419
2024-01-04 17:15:00,44196.511719,44245.570312,44128.683594,44128.683594,44128.683594,0,43393.163065,2,39.844647,43.047066,12.512765
2024-01-04 17:30:00,44144.972656,44172.367188,43931.804688,43931.804688,43931.804688,63057920,43403.506682,2,39.317199,38.596834,19.679967


In [24]:
# Define a function to generate the trend signal based on ADX
def generate_trend_signal(data, threshold=40):
    trend_signal = []
    for i in range(len(data)):
        if data['ADX'][i] > threshold:
            if data['DMP'][i] > data['DMN'][i]:
                trend_signal.append(2)  # Confirmed Uptrend
            else:
                trend_signal.append(1)  # Confirmed Downtrend
        else:
            trend_signal.append(0)  # No confirmed trend
    return trend_signal

In [25]:
# Apply the function to generate the trend signal column
data = data.rename(columns=lambda x: x[:-3] if x.startswith('ADX') else x)
data = data.rename(columns=lambda x: x[:-3] if x.startswith('DM') else x)

data['Trend Signal'] = generate_trend_signal(data)

In [26]:
data[data['Trend Signal']!=0] #only confirmed signals not 0 that is nor uptrend nor downtrend

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category,ADX,DMP,DMN,Trend Signal
Datetime,,,,,,,,,,,,
2023-12-30 16:15:00,42549.226562,42584.125000,42491.968750,42491.968750,42491.968750,0,41947.528198,2,42.333907,55.127872,8.510528,2
2023-12-30 16:30:00,42489.578125,42551.128906,42487.835938,42487.835938,42487.835938,0,41947.528198,2,44.474282,52.303057,8.409026,2
2023-12-30 16:45:00,42456.136719,42457.875000,42393.593750,42407.230469,42407.230469,0,41947.528198,2,44.994689,48.331813,15.363305,2
2023-12-30 17:00:00,42398.785156,42425.082031,42307.847656,42309.082031,42309.082031,0,41947.528198,2,44.344202,43.869533,20.697661,2
2023-12-30 17:15:00,42300.746094,42539.035156,42282.554688,42530.804688,42530.804688,74414080,41984.267551,2,44.336400,43.969587,16.999793,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-03 16:45:00,43008.812500,43119.843750,42912.054688,43006.785156,43006.785156,173129728,42731.052838,0,45.878054,21.313126,32.989603,1
2024-01-03 17:00:00,43042.050781,43056.906250,42898.402344,42939.621094,42939.621094,77045760,42732.022658,2,44.168826,20.571526,32.141417,1
2024-01-03 17:15:00,42956.820312,42956.820312,42726.460938,42828.023438,42828.023438,261906432,42733.482990,2,42.980821,19.509073,34.336358,1


In [27]:
data['Confirmed Signal'] = data.apply(lambda row: row['Category'] if row['Category'] == row['Trend Signal'] else 0, axis=1) # we combine moving average and ADX

In [28]:
#check if there are error 
data[data['Confirmed Signal']!=0]
 
 #in the video uses
# data[(data['Category']!= data['Trend Signal']) &   (data[data['Confirmed Signal']!=0)]  

,Open,High,Low,Close,Adj Close,Volume,VWAP_D,Category,ADX,DMP,DMN,Trend Signal,Confirmed Signal
Datetime,,,,,,,,,,,,,
2023-12-30 16:15:00,42549.226562,42584.125000,42491.968750,42491.968750,42491.968750,0,41947.528198,2,42.333907,55.127872,8.510528,2,2.0
2023-12-30 16:30:00,42489.578125,42551.128906,42487.835938,42487.835938,42487.835938,0,41947.528198,2,44.474282,52.303057,8.409026,2,2.0
2023-12-30 16:45:00,42456.136719,42457.875000,42393.593750,42407.230469,42407.230469,0,41947.528198,2,44.994689,48.331813,15.363305,2,2.0
2023-12-30 17:00:00,42398.785156,42425.082031,42307.847656,42309.082031,42309.082031,0,41947.528198,2,44.344202,43.869533,20.697661,2,2.0
2023-12-30 17:15:00,42300.746094,42539.035156,42282.554688,42530.804688,42530.804688,74414080,41984.267551,2,44.336400,43.969587,16.999793,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-03 14:45:00,42384.125000,42384.125000,42232.351562,42250.304688,42250.304688,89948160,42672.573688,1,55.615999,7.254564,50.297546,1,1.0
2024-01-03 15:00:00,42240.496094,42507.457031,42240.496094,42463.277344,42463.277344,103407616,42670.818561,1,56.399912,9.504293,47.391940,1,1.0
2024-01-03 15:15:00,42454.453125,42501.914062,42355.437500,42465.183594,42465.183594,0,42670.818561,1,57.127832,9.190576,45.827632,1,1.0


In [29]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['VWAP_D'], mode='lines', name='VWAP', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()